# **Machine Learning Training**

### Initial Imports:

In [32]:
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 300)

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from joblib import dump, load # Save Pre-Trained Model Using Joblib
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Set Initial Random State
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [9]:
ticker = "TSLA"

#### Read in CSV as Pandas DataFrame:

In [7]:
# Set path to CSV and read in CSV
csv_path = Path('Returns and Signals.csv')
return_signal_df=pd.read_csv(csv_path)
# Set index as datetime object and drop extraneous columns
return_signal_df.set_index(pd.to_datetime(return_signal_df['Date'], infer_datetime_format=True), inplace=True)
return_signal_df.drop(columns=['Date'], inplace=True)
return_signal_df.head()

,TSLA_Adj Close,TSLA_Volume,TSLA_Returns,ema_crossover_signal,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal
Date,,,,,,,,
2019-01-02,62.023998,58293000.0,-0.068149,-1.0,0.0,0.0,-1.0,0.0
2019-01-03,60.071999,34826000.0,-0.031472,-1.0,1.0,0.0,-1.0,-1.0
2019-01-04,63.537998,36970500.0,0.057697,-1.0,1.0,0.0,-1.0,0.0
2019-01-07,66.991997,37756000.0,0.054361,1.0,-1.0,0.0,1.0,0.0
2019-01-08,67.070000,35042500.0,0.001164,1.0,1.0,0.0,1.0,0.0


#### Check Data Quality:

In [8]:
return_signal_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 504 entries, 2019-01-02 to 2020-12-30
Data columns (total 8 columns):
TSLA_Adj Close          504 non-null float64
TSLA_Volume             504 non-null float64
TSLA_Returns            504 non-null float64
ema_crossover_signal    504 non-null float64
vol_trend_signal        504 non-null float64
bollinger_signal        504 non-null float64
obv_crossover_signal    504 non-null float64
sentiment_signal        504 non-null float64
dtypes: float64(8)
memory usage: 35.4 KB


### Creating X and Dependent Variables:

In [17]:
def variables(return_signal_df):
    '''
    this func takes in return_signal_df dataframe,
    then shiting X variables down (to the future) by 1,
    then constructing a new column for the dependent variable
    return modified return_signal_df.
    '''
    # Set X Variables/Features 
    global x_var_list
    x_var_list = ['ema_crossover_signal','vol_trend_signal','bollinger_signal','obv_crossover_signal','sentiment_signal'] # as a filter
    # Shift 1 down as signals comes after prices 
    return_signal_df[x_var_list] = return_signal_df[x_var_list].shift(1)
    # Construct the dependent variable where if daily return is greater than 0, then 1, else, 0.
    return_signal_df['Positive Return'] = np.where(return_signal_df[f'{ticker}_Returns'] > 0, 1.0, 0.0)
    return_signal_df.dropna(inplace=True)
    return return_signal_df

In [18]:
return_signal_df_modified = variables(return_signal_df)
return_signal_df_modified.head()

,TSLA_Adj Close,TSLA_Volume,TSLA_Returns,ema_crossover_signal,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal,Positive Return
Date,,,,,,,,,
2019-01-07,66.991997,37756000.0,0.054361,-1.0,0.0,0.0,-1.0,0.0,1.0
2019-01-08,67.070000,35042500.0,0.001164,-1.0,1.0,0.0,-1.0,-1.0,1.0
2019-01-09,67.706001,27164500.0,0.009483,-1.0,1.0,0.0,-1.0,0.0,1.0
2019-01-10,68.994003,30282000.0,0.019023,1.0,-1.0,0.0,1.0,0.0,1.0
2019-01-11,69.452003,25195500.0,0.006638,1.0,1.0,0.0,1.0,0.0,1.0


### Separate X and Y Training Datasets:
80% (training) 20% (testing) split

In [19]:
def separate(return_signal_df_modified):
    '''
    thif func takes in return_signal_df_modified dataframe,
    split the data into trian (80%) and test (20%),
    then construct and return trian and test data sets for 
    the X and dependent variables.
    '''
    split = int(0.8 * len(return_signal_df_modified))
    # X Variable
    x_train = return_signal_df_modified[x_var_list][:split]
    x_test = return_signal_df_modified[x_var_list][split:]
    # Dependent Variable
    y_train = return_signal_df_modified['Positive Return'][:split]
    y_test = return_signal_df_modified['Positive Return'][split:]
    return x_train, x_test, y_train, y_test

In [20]:
x_train, x_test, y_train, y_test = separate(return_signal_df_modified)

In [24]:
x_train.head(2)

,ema_crossover_signal,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal
Date,,,,,
2019-01-07,-1.0,0.0,0.0,-1.0,0.0
2019-01-08,-1.0,1.0,0.0,-1.0,-1.0


In [23]:
x_test.head(2)

,ema_crossover_signal,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal
Date,,,,,
2020-08-07,1.0,1.0,0.0,1.0,0.0
2020-08-10,1.0,1.0,0.0,1.0,0.0


In [22]:
y_train.head(2)

Date
2019-01-07    1.0
2019-01-08    1.0
Name: Positive Return, dtype: float64

In [21]:
y_test.head(3)

Date
2020-08-07    0.0
2020-08-10    0.0
2020-08-11    0.0
Name: Positive Return, dtype: float64

### Machine Learning Models Training:

#### 1. Random Forest Model:

**Hyperparameters:**

In [29]:
# 100 trees and 3 steps
n_estimators=100
max_depth=3

In [60]:
def RandomForest(x_train, y_train, x_test, y_test):
    '''
    thif func takes in x_train, y_train, x_test, y_test,
    runing Random Forest Classification model,
    return a dataframe which contains actual vs. predicted y data,
    print accuracy score for traning and testing,
    print the classification report for the test data,
    and save the model by using Joblib.
    '''
    # Fit the model using just the training set:
    model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
    model.fit(x_train, y_train)
    # Make a prediction of "y" values from the x test dataset
    predictions = model.predict(x_test)
    # Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
    result = y_test.to_frame()
    result["Predicted Value"] = predictions
    # Save the pre-trained model
    dump(model, 'random_forest_model.joblib')
    # Print performance metrics:
    print("Accuracy score (training): {0:.3f}".format(model.score(x_train,y_train)))
    print("Accuracy score (testing): {0:.3f}".format(model.score(x_test,y_test)))
    print(" ")
    print("Classification Report for the test data:")
    print(classification_report(y_test, predictions))
    return result

In [61]:
rf_result = RandomForest(x_train, y_train, x_test, y_test)
rf_result.head()

Accuracy score (training): 0.578
Accuracy score (testing): 0.594
 
Classification Report for the test data:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        41
         1.0       0.59      1.00      0.75        60

    accuracy                           0.59       101
   macro avg       0.30      0.50      0.37       101
weighted avg       0.35      0.59      0.44       101



,Positive Return,Predicted Value
Date,,
2020-08-07,0.0,1.0
2020-08-10,0.0,1.0
2020-08-11,0.0,1.0
2020-08-12,1.0,1.0
2020-08-13,1.0,1.0


#### 2. Gradient Boosting Model:

**Hyperparameters:**

In [62]:
# small learning rate means each tree learns a little from previous tree, 
# and learn more information from the new data.
learning_rate = 0.5
n_estimators = 20
max_features = 5
max_depth = 3

In [80]:
def GradientBoost(x_train, y_train, x_test, y_test):
    '''
    thif func takes in x_train, y_train, x_test, y_test,
    runing Gradient Boost Trees Classification model,
    return a dataframe which contains actual vs. predicted y data,
    print accuracy score for traning and testing,
    print the classification report for the test data,
    and save the model by using Joblib.
    '''
    # Fit the model using just the training set:
    model = GradientBoostingClassifier(n_estimators,learning_rate,max_features,max_depth,random_state=0)
    model.fit(x_train, y_train.values)
    # Make a prediction of "y" values from the x test dataset
    predictions = model.predict(x_test)
    
    
    
    # Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
    result = y_test.values.to_frame()
    result["Predicted Value"] = predictions
    # Save the pre-trained model
    dump(model, 'gradient_boost_model.joblib')
    # Print performance metrics:
    print("Accuracy score (training): {0:.3f}".format(model.score(x_train,y_train)))
    print("Accuracy score (testing): {0:.3f}".format(model.score(x_test,y_test)))
    print(" ")
    print("Classification Report for the test data:")
    print(classification_report(y_test, predictions))
    
    return result

In [83]:
model = GradientBoostingClassifier(n_estimators,learning_rate,max_features,max_depth,random_state=0)

In [82]:
model.fit(x_train, y_train.values)
# Make a prediction of "y" values from the x test dataset
predictions = model.predict(x_test)
predictions

ValueError: Unknown format code 's' for object of type 'int'

In [81]:
rf_result = GradientBoost(x_train, y_train, x_test, y_test)
rf_result.head()

ValueError: Unknown format code 's' for object of type 'int'